### Perimeter Institute Nov 16th
### 2018 computational physics course
# Exact Diagonalization
 
Guifre Vidal  

# IJulia nb2: quantum Ising model

In [ ]:
using LinearAlgebra

In [ ]:
# Pauli matrices X and Z, and identity matrix I
X = [0. 1; 1 0]
Z = [1. 0; 0 -1]
E = Matrix(1.0I, 2,2) # or E = [1 0; 0 1], or E = diagm(0=>[1; 1]), or diagm(0=>ones(2))
XX = kron(X,X)
display(Z)
display(X)
display(E)
display(XX)

### Hamiltonian for $N$ spins (use $N\leq 10$) 

In [ ]:
# Ising Hamiltonian for N spins  with exact diagonalization
N = 3
theta = pi/4 # magnetic field
HXX = XX
HZ = kron(Z,E) + kron(E,Z)
for n = 3:N
    HXX = kron(HXX,E)+kron(diagm(0=>ones(2^(n-2))), XX)
    HZ =  kron(HZ,E) + kron(diagm(0=>ones(2^(n-1))),Z)
end
HXX = HXX + kron(X,kron(diagm(0=>ones(2^(N-2))),X))
H = -cos(theta)*HXX - sin(theta)*HZ

display(H)

###  magnetic field angle $\theta \in [0,\pi/2]$ 

In [ ]:
N_theta=21  # number of values of magnetic field
d_theta = pi/(2*(N_theta-1)) # increments of theta
theta_val = collect(0:d_theta:pi/2) # list of magnetic field angle theta [0, pi/2] to be considered
Energies = zeros(2^3,N_theta)
for i=1:N_theta
    H = -cos(theta_val[i])*HXX - sin(theta_val[i])*HZ # build the Hamiltonian
    D,U = eigen(H)          # diagonalize the Hamiltonian
    Energies[:,i] = D
end
[theta_val Energies[1,:]]

### Let us plot this!

In [ ]:
using PyPlot

In [ ]:
figure("My_figure",figsize=(8,3))
grid("on")
for i in 1:2^N
    plot(2*theta_val/pi,vec(Energies[i,:]), marker=".")
end
title("Energy spectrum of quantum Ising model")
xlabel("magnetic field theta")
ylabel("Energy levels");


# Now it's your turn! (1/3)
### 1) write a function buildH that builds the Hamiltonian for $N$ spins (use only for $N\leq 10$)

In [ ]:
# function buildH takes as input N (number of spins) and theta (magnetic field angle) and output H (Hamiltonian)
function buildH(N::Int64,theta::Float64)::Array{Float64,2} 
    #
    # complete this! 
    #
    return H
end

buildH(3,pi/4) # test for N=3 and critical angle pi/4

### 2) plot the energy spectrum as a function of angle theta, $\theta \in [0,\pi/2]$, for $N=8$ spins

# Let us continue: 
### Translation operator $T$, with $[H,T]=0$ (translation invariance)

In [ ]:
# Let us quickly rebuild the Hamiltonian for N=3 spins and magnetic field angle theta = pi/4
EE = kron(E,E)
HZ = kron(Z,EE) + kron(E,kron(Z,E)) + kron(EE,Z)
HXX = kron(XX,E)+kron(E,XX)+kron(X,kron(E,X))
H = -(HXX+HZ)/sqrt(2)
 
# The translation operator T is
SWAP = [1 0 0 0; 0 0 1 0; 0 1 0 0; 0 0 0 1]
N = 3
T = copy(SWAP)
for n=3:N
    SWAPn = kron(diagm(0=>ones(2^(n-2))), SWAP)
    T =  SWAPn*kron(T,E)
end
#display(SWAP)
#display(SWAP^2)
display(T)
#display(T^3)
#display(H3*T-T*H3)

### Let us diagonalize $H$ and $T$ simultaneously, 
### $H |\Psi_i\rangle = e_i |\Psi_i\rangle$  and   $T |\Psi_i\rangle = \exp(ik_i) |\Psi_i\rangle$

In [ ]:
HT = H + 0.0001*T 
D,U = eigen(HT);

### Let us compute and plot energies $e_i$ as a function of momenta $k_i$

In [ ]:
e = real(diag(U'*H*U))  
k = angle.(diag(U'*T*U))

perm = sortperm(e) # re-order eigenvalues according to increasing energies e
e = e[perm]
k = k[perm] 

[e k]

In [ ]:
figure("low_energies",figsize=(8,3))

grid("on") # Create a grid on the axis
title("low energies spin chain")
#ax = gca()
#ax[:set_xlim]([-3,3])
#ax[:set_ylim]([-0.2,2.8])
xlabel("momentum")
ylabel("energy")
for i in 1:size(e,1)
    plot(k[i],e[i], marker = "s", markersize = 8, color = "y")
end


# It's your turn again! (2/3)
### 3) write a function buildT that builds the translation operator $T$ for $N$ spins (use only for $N\leq 10$)

In [ ]:
# function buildT takes as input N (number of spins) and outputs T (translatio operator)
function buildT(N::Int64)::Array{Float64,2} 
        #
        # complete this
        #
    return T
end

buildT(3) # test for N=3 and critical angle pi/4

### 4) compute and plot energies $e_i$ as a function of momenta $k_i$ for $N=8$ spins and $\theta=\pi/4$

# Let us continue:
### We add the parity operator $P$, with $[H,P]=[T,P]=0$ (parity conservation)

In [ ]:
# Let us quickly rebuild the Hamiltonian for N=3 spins and magnetic field angle theta = pi/4
EE = kron(E,E)
HZ = kron(Z,EE) + kron(E,kron(Z,E)) + kron(EE,Z)
HXX = kron(XX,E)+kron(E,XX)+kron(X,kron(E,X))
H = -(HXX+HZ)/sqrt(2)
# and the translation operator for N=3 
T = kron(E,SWAP)*kron(SWAP,E)

# The parity operator P is
Z = [1 0; 0 -1]
N=3
P = copy(Z)
for i=2:N
    P = kron(P,Z)
end
#display(Z)
display(P)
#display(P^2)
#display(norm(H3*P-P*H3))
#display(norm(T*P-P*T))

### Let us diagonalize $H$, $T$, and $P$ simultaneously, 
### $H |\Psi_i\rangle = e_i |\Psi_i\rangle$,   $~~~T |\Psi_i\rangle = \exp(ik_i) |\Psi_i\rangle$, and $~~~Z |\Psi_i\rangle = (-1)^p |\Psi_i\rangle$

In [ ]:
HTP = H + 0.0001*T + im*0.01*P
D,U = eigen(HTP)

### Let us compute and plot energies $e_i$, momenta $k_i$, and parity $p_i$

In [ ]:
e = real(diag(U'*H*U)) 
k = angle.(diag(U'*T*U))
p = real(diag(U'*P*U))
p = round.((1 .- p)/2) 
perm = sortperm(e) # re-order eigenvalues according to E
e = e[perm]
k = k[perm] 
p = p[perm]
[e k p]


In [ ]:
figure("low_energies",figsize=(8,3))

subplot(121) # Create the 1st axis of a 2x2 arrax of axes
grid("on") # Create a grid on the axis
title("Even Parity, p=0")
xlabel("momentum")
ylabel("energy")

subplot(122) # Create the 1st axis of a 2x2 arrax of axes
grid("on") # Create a grid on the axis
title("Even Parity, p=1")
xlabel("momentum")
ylabel("energy")

for i in 1:size(e,1)
    if p[i] == 0
        subplot(121)
        plot(k[i],e[i], marker = "s", color = "b")
    elseif p[i] == 1
        subplot(122)
        plot(k[i],e[i], marker = "o", color = "k")
    end
end   

# It's your turn for the last time! (3/3)
### 5) write a function buildP that builds the parity operator $P$ for $N$ spins (use only for $N\leq 10$)

In [ ]:
# function buildP takes as input N (number of spins) and outputs P (parity operator)
function buildP(N::Int64)::Array{Float64,2} 
        #
        # complete this
        #
    return P
end

buildP(3) # test for N=3 and critical angle pi/4

### 6) compute and plot energies $e_i$ as a function of momenta $k_i$ for $N=8$ spins and $\theta=\pi/4$, for parity $p=0,1$

### Function that builds the Hamiltonian for $N$ spins (use $N\leq 10$) 

# Good job!!! (maybe...)